In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Задача: разработать сервис, который будет предсказывать стоимость домов, основываясь на истории предложений.

In [ ]:
pip install PrettyTable

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from ast import literal_eval
from sklearn import preprocessing
from sklearn.preprocessing import (StandardScaler, MinMaxScaler)
from sklearn.model_selection import (train_test_split, KFold, StratifiedKFold)
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, StackingRegressor)
from sklearn.tree import DecisionTreeRegressor
from catboost import CatBoostRegressor
import shap
from xgboost import XGBRegressor
from prettytable import PrettyTable

from sklearn.metrics import f1_score
from sklearn.metrics import (mean_squared_error, mean_absolute_error)
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
# зафиксирую значение RANDOM_SEED, чтобы эксперименты были воспроизводимыми:

RANDOM_SEED = 42

In [ ]:
# Загружаю датасет

data = pd.read_csv('/kaggle/input/diplomdata/data.csv')
data.head()

In [ ]:
data.columns

In [ ]:
data.info()

 Создание бейзлайна.
 
 Буду обучать модель CatBoost и измерять точность на k-fold кросс-валидации после преобразования принаков, чтобы понять, какие из преобразований действительно улучшают модель. 

Посмотрю на целевую переменную

In [ ]:
data['target'].value_counts()

In [ ]:
# Преобразую целевую переменную для использования в модели:

data['target'] = data['target'].apply(lambda x: str(x).replace('$',''))
data['target'] = data['target'].apply(lambda x: str(x).replace('+',''))
data['target'] = data['target'].apply(lambda x: str(x).replace(',',''))
data['target'] = data['target'].apply(lambda x: str(x).replace('/mo','000'))
data['target'] = data['target'].apply(lambda x: str(x).replace('1,215 - 1,437','1300000'))
data['target'] = data['target'].apply(lambda x: str(x).replace('nan','225000'))
data['target'] = data['target'].apply(lambda x: str(x).replace('1215 - 1437000','1437000'))

In [ ]:
X2 = data.drop('target',axis = 1).fillna('No info')
y2 = data['target']

skf = StratifiedKFold(n_splits=4, random_state=42, shuffle=True)
cat_boost_mape_values = []
mae_values = []
rmse_values = []

for train_index, test_index in skf.split(X2, y2):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train2, X_test2 = X2[X2.index.isin(train_index)], X2[X2.index.isin(test_index)]
    y_train2, y_test2 = y2[y2.index.isin(train_index)], y2[y2.index.isin(test_index)]
    

    model = CatBoostRegressor(iterations=300)
    model.fit(X_train2, y_train2,eval_set=(X_test2, y_test2), verbose=100, cat_features=['status', 'private pool', 'propertyType', 'street', 'baths', 'homeFacts', 'fireplace', 'city', 'schools', 'sqft', 'zipcode', 'beds', 'state', 'stories', 'mls-id', 'PrivatePool', 'MlsId'])



    y_pred2 = model.predict(X_test2)

    print(np.sqrt(mean_squared_error(y_test2, y_pred2)))

    cat_boost_mape_value = (mean_absolute_percentage_error(y_test2, y_pred2))
    cat_boost_mape_values.append(cat_boost_mape_value)
    print(cat_boost_mape_value)
    mae_value = (mean_absolute_error(y_test2, y_pred2))
    mae_values.append(mae_value)
    rmse_value = np.sqrt(mean_squared_error(y_test2, y_pred2))
    rmse_values.append(rmse_value)
    
print(f"The MAPE mertic for the default CatBoost model using 4-fold CV is: {(np.mean(cat_boost_mape_values)):0.2f}%.")
print(f"The MAE mertic for default CatBoost model: {(np.mean(mae_values)):0.2f}")
print(f"The RMSE default CatBoost model: {(np.mean(rmse_values)):0.2f}")

The MAPE mertic for the default CatBoost model using 4-fold CV is: 15.33%.

The MAE mertic for default CatBoost model: 300495.67

The RMSE default CatBoost model: 1298697.18

In [ ]:
# Построю график важности признаков для модели

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X2)

In [ ]:
shap.summary_plot(shap_values, X2)

In [ ]:
# Посмотрю на количество пустых значений в столбцах

nan_count=100*data.isna().sum().sort_values(ascending=False)/data.shape[0]
fig=px.bar(x=nan_count.index,y=nan_count.values, labels={"y": "Nan ammount (%)","x": "Feature"})
fig.show()

In [ ]:
# Столбцы 'privat pool' и 'mis-id' имеют более чем 80% пропусков и вносят наименьший вклад в модель. Попробуем их удалить

data.drop('private pool', axis=1, inplace=True)

In [ ]:
X3 = data.drop('target',axis = 1).fillna('No info')
y3 = data['target']

skf = StratifiedKFold(n_splits=4, random_state=42, shuffle=True)
cat_boost_mape_values = []
mae_values = []
rmse_values = []

for train_index, test_index in skf.split(X3, y3):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train3, X_test3 = X3[X3.index.isin(train_index)], X3[X3.index.isin(test_index)]
    y_train3, y_test3 = y3[y3.index.isin(train_index)], y3[y3.index.isin(test_index)]
    

    model2 = CatBoostRegressor(iterations=300)
    model2.fit(X_train3, y_train3,eval_set=(X_test3, y_test3), verbose=100, cat_features=['mls-id', 'status', 'propertyType', 'street', 'baths', 'homeFacts', 'fireplace', 'city', 'schools', 'sqft', 'zipcode', 'beds', 'state', 'stories', 'PrivatePool', 'MlsId'])



    y_pred3 = model2.predict(X_test3)

    print(np.sqrt(mean_squared_error(y_test3, y_pred3)))

    cat_boost_mape_value = (mean_absolute_percentage_error(y_test3, y_pred3))
    cat_boost_mape_values.append(cat_boost_mape_value)
    print(cat_boost_mape_value)
    mae_value = (mean_absolute_error(y_test3, y_pred3))
    mae_values.append(mae_value)
    rmse_value = np.sqrt(mean_squared_error(y_test3, y_pred3))
    rmse_values.append(rmse_value)
    
print(f"The MAPE mertic for the default CatBoost model using 4-fold CV is: {(np.mean(cat_boost_mape_values)):0.2f}%.")
print(f"The MAE mertic for default CatBoost model: {(np.mean(mae_values)):0.2f}")
print(f"The RMSE default CatBoost model: {(np.mean(rmse_values)):0.2f}")


The MAPE mertic for the default CatBoost model using 4-fold CV is: 14.60%.

The MAE mertic for default CatBoost model: 299712.70

The RMSE default CatBoost model: 1291909.98

Метрика улучшилась, признак можно удалить

In [ ]:
# data.drop('mls-id', axis=1, inplace=True)

In [ ]:
'''
X3 = data.drop('target',axis = 1).fillna('No info')
y3 = data['target']

skf = StratifiedKFold(n_splits=4, random_state=42, shuffle=True)
cat_boost_mape_values = []
mae_values = []
rmse_values = []

for train_index, test_index in skf.split(X3, y3):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train3, X_test3 = X3[X3.index.isin(train_index)], X3[X3.index.isin(test_index)]
    y_train3, y_test3 = y3[y3.index.isin(train_index)], y3[y3.index.isin(test_index)]
    

    model2 = CatBoostRegressor(iterations=300)
    model2.fit(X_train3, y_train3,eval_set=(X_test3, y_test3), verbose=100, cat_features=['status', 'propertyType', 'street', 'baths', 'homeFacts', 'fireplace', 'city', 'schools', 'sqft', 'zipcode', 'beds', 'state', 'stories', 'PrivatePool', 'MlsId'])


    y_pred3 = model2.predict(X_test3)

    print(np.sqrt(mean_squared_error(y_test3, y_pred3)))

    cat_boost_mape_value = (mean_absolute_percentage_error(y_test3, y_pred3))
    cat_boost_mape_values.append(cat_boost_mape_value)
    print(cat_boost_mape_value)
    mae_value = (mean_absolute_error(y_test3, y_pred3))
    mae_values.append(mae_value)
    rmse_value = np.sqrt(mean_squared_error(y_test3, y_pred3))
    rmse_values.append(rmse_value)
    
print(f"The MAPE mertic for the default CatBoost model using 4-fold CV is: {(np.mean(cat_boost_mape_values)):0.2f}%.")
print(f"The MAE mertic for default CatBoost model: {(np.mean(mae_values)):0.2f}")
print(f"The RMSE default CatBoost model: {(np.mean(rmse_values)):0.2f}")
'''

The MAPE mertic for the default CatBoost model using 4-fold CV is: 15.35%.

The MAE mertic for default CatBoost model: 298805.77

The RMSE default CatBoost model: 1290314.38

Показания метрики ухудшилось, столбец нужно оставить

In [ ]:
data.info()

Преобразования признаков ухудшают метрики модели. Исключение - 'homeFacts' и 'schools'

# homeFacts

In [ ]:
data['homeFacts']

In [ ]:
# Данный столбец содержит 7 отдельных признаков. Информация о площади и цене в датасете уже есть. 
# Добавление в датасет 'Remodeled year', 'Heating' и 'Cooling' ухудшает метрику,  'Parking' и 'Year built' делает лучше. 

data['homeFacts'][1]

In [ ]:
ser = pd.Series(data['homeFacts']) 

ser.head(10) 

In [ ]:
ser[0][35:39]

In [ ]:
year = []
for j in range(0,377185):
    year.append(ser[j][35:39])

In [ ]:
year[:10]

In [ ]:
data['year'] = year

In [ ]:
data['year'].value_counts().head(20)

In [ ]:
data['year'].unique()

In [ ]:
def year_cod(status):
    if "', '" in status: return '0'
    elif 'one,' in status: return '0'
    elif 'No D' in status: return '0'
    elif "1', " in status: return '0'
    else: 
        return status
    
data['year'] = data['year'].apply(lambda x: year_cod(x))
data['year'].value_counts()

Теперь проанализирую  данные о парковке

In [ ]:
from ast import literal_eval
def funk(x):
    homefact_dict = literal_eval(x)
    homefact_lst = homefact_dict['atAGlanceFacts'][4].get('factValue')
    if homefact_lst:
        return str(homefact_lst)
    else:
        return np.nan
Parking = list(data[:].homeFacts.apply(funk))
Parking[:10]

In [ ]:
data['Parking'] = Parking  

In [ ]:
data['Parking'] = data['Parking'].apply(lambda x: str(x).lower()) # приведу данные к нижнему регистру

In [ ]:
data['Parking'].value_counts().head(30)

In [ ]:
def parking_cod(status):
    if '1' in status: return 'one'
    elif '2' in status: return 'two'
    elif '3' in status: return 'three'
    elif '4' in status: return 'four'
    elif 'attached' in status: return 'attached'
    elif 'detached' in status: return 'detached'
    elif 'carport' in status: return 'carport'
    elif 'no data' in status: return 'no info'
    elif '5' in status: return 'five'
    elif 'parking' in status: return 'parking'
    elif 'on street' in status: return 'on street'
    elif 'off street' in status: return 'off street'
    elif 'none' in status: return 'zero'
    elif '0' in status: return 'zero'
    elif 'garage' in status: return 'one'
    elif 'nan' in status: return 'no info'
    else: 
        return 'other'
    
data['Parking'] = data['Parking'].apply(lambda x: parking_cod(x))
data['Parking'].value_counts()

In [ ]:
data.drop('homeFacts', axis=1, inplace=True)

In [ ]:
data.info()

In [ ]:
X3 = data.drop('target',axis = 1).fillna('No info')
y3 = data['target']

skf = StratifiedKFold(n_splits=4, random_state=42, shuffle=True)
cat_boost_mape_values = []
mae_values = []
rmse_values = []

for train_index, test_index in skf.split(X3, y3):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train3, X_test3 = X3[X3.index.isin(train_index)], X3[X3.index.isin(test_index)]
    y_train3, y_test3 = y3[y3.index.isin(train_index)], y3[y3.index.isin(test_index)]
    

    model2 = CatBoostRegressor(iterations=300)
    model2.fit(X_train3, y_train3,eval_set=(X_test3, y_test3), verbose=100, cat_features=['Parking',  'mls-id', 'year','status','propertyType', 'street', 'baths',  'fireplace', 'city', 'schools', 'sqft', 'zipcode', 'beds', 'state', 'stories', 'PrivatePool', 'MlsId'])


    y_pred3 = model2.predict(X_test3)

    print(np.sqrt(mean_squared_error(y_test3, y_pred3)))

    cat_boost_mape_value = (mean_absolute_percentage_error(y_test3, y_pred3))
    cat_boost_mape_values.append(cat_boost_mape_value)
    print(cat_boost_mape_value)
    mae_value = (mean_absolute_error(y_test3, y_pred3))
    mae_values.append(mae_value)
    rmse_value = np.sqrt(mean_squared_error(y_test3, y_pred3))
    rmse_values.append(rmse_value)
    
print(f"The MAPE mertic for the default CatBoost model using 4-fold CV is: {(np.mean(cat_boost_mape_values)):0.2f}%.")
print(f"The MAE mertic for default CatBoost model: {(np.mean(mae_values)):0.2f}")
print(f"The RMSE default CatBoost model: {(np.mean(rmse_values)):0.2f}")


The MAPE mertic for the default CatBoost model using 4-fold CV is: 12.76%.

The MAE mertic for default CatBoost model: 300974.52

The RMSE default CatBoost model: 1292707.73

# schools

In [ ]:
data['schools']

In [ ]:
type(data['schools'])

In [ ]:
# здесь наиболее интересными данными мне кажутся рейтинг школы и расстояние до школы. Попробую вычислить средний рейтинг и среднее расстояние.

data['schools'][1]

In [ ]:
def funk(x):
    Distance = literal_eval(x)
    Distance_list = Distance[0]['data'].get('Distance')
    if Distance_list:
        return Distance_list
    else:
        return '0'
    
Distance = list(data[:].schools.apply(funk))
Distance[:10]

In [ ]:
data['Distance'] = Distance

In [ ]:
data['Distance'][:10]

In [ ]:
average_distance_school = []

In [ ]:
for x in range(0,377185):
   
    l = data['Distance'][x]
    l1=[]
    for x in l:
        x =str(x).replace('mi','').replace('[','').replace(']','')
        l1.append(x)
    floatlist = [float(elem) for elem in l1]
    average_distance_school.append(sum(floatlist)/len(floatlist))
    
       

In [ ]:
average_distance_school[:10]

In [ ]:
len(average_distance_school)

In [ ]:
data['average_distance_school'] = average_distance_school
data['average_distance_school'] = data['average_distance_school'].apply(lambda x: round(x,2))

In [ ]:
data['average_distance_school'].value_counts().head(30)

In [ ]:
data.average_distance_school.isna().sum()

In [ ]:
def funk(x):
    rating = literal_eval(x)
    rating_list = rating[0].get('rating')
    if rating_list:
        return rating_list
    else:
        return '0'
    
rating_school = list(data[:].schools.apply(funk))
rating_school[:10]

In [ ]:
data['rating_school'] = rating_school

In [ ]:
data['rating_school'][:10]

In [ ]:
 average_rating_school = []

In [ ]:
for x in range(0,377185):
   
    m = data['rating_school'][x]
    m1=[]
    for x in m:
        x =str(x).replace('1/10','1').replace('2/10','2').replace('4/10','4').replace('5/10','5').replace('6/10','6').replace('7/10','7').replace('8/10','8').replace('9/10','9').replace('10/10','10').replace('None/10','0').replace('NA','0').replace('NR','0').replace('3/10','3').replace('[','').replace(']','').replace('','0')
        m1.append(x)
    floatlist = [float(elem) for elem in m1]
    average_rating_school.append((sum(floatlist)/len(floatlist))/10)

In [ ]:
data['average_rating_school'] = average_rating_school
data['average_rating_school'] = data['average_rating_school'].apply(lambda x: round(x,2))

In [ ]:
data['average_rating_school'].value_counts()

In [ ]:
data.info()

In [ ]:
data.drop('schools', axis=1, inplace=True)
data.drop('Distance', axis=1, inplace=True)
data.drop('rating_school', axis=1, inplace=True)

In [ ]:
X3 = data.drop('target',axis = 1).fillna('No info')
y3 = data['target']

skf = StratifiedKFold(n_splits=4, random_state=42, shuffle=True)
cat_boost_mape_values = []
mae_values = []
rmse_values = []

for train_index, test_index in skf.split(X3, y3):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train3, X_test3 = X3[X3.index.isin(train_index)], X3[X3.index.isin(test_index)]
    y_train3, y_test3 = y3[y3.index.isin(train_index)], y3[y3.index.isin(test_index)]
    
    model2 = CatBoostRegressor(iterations=300)
    model2.fit(X_train3, y_train3,eval_set=(X_test3, y_test3), verbose=100, cat_features=['mls-id', 'year','Parking','status','propertyType', 'street', 'baths',  'fireplace', 'city', 'sqft', 'zipcode', 'beds', 'state', 'stories', 'PrivatePool', 'MlsId'])


    y_pred3 = model2.predict(X_test3)

    print(np.sqrt(mean_squared_error(y_test3, y_pred3)))

    cat_boost_mape_value = (mean_absolute_percentage_error(y_test3, y_pred3))
    cat_boost_mape_values.append(cat_boost_mape_value)
    print(cat_boost_mape_value)
    mae_value = (mean_absolute_error(y_test3, y_pred3))
    mae_values.append(mae_value)
    rmse_value = np.sqrt(mean_squared_error(y_test3, y_pred3))
    rmse_values.append(rmse_value)
    
print(f"The MAPE mertic for the default CatBoost model using 4-fold CV is: {(np.mean(cat_boost_mape_values)):0.2f}%.")
print(f"The MAE mertic for default CatBoost model: {(np.mean(mae_values)):0.2f}")
print(f"The RMSE default CatBoost model: {(np.mean(rmse_values)):0.2f}")


The MAPE mertic for the default CatBoost model using 4-fold CV is: 12.37%.


In [ ]:
data.head()

In [ ]:
df = data.copy()

In [ ]:
# Несколько столбцов можно перевести из object в int

df['average_rating_school'] = df['average_rating_school'].astype(int)
df['year'] = df['year'].astype(int)
df['target'] = df['target'].astype(int)

In [ ]:
df.info()

In [ ]:
from sklearn import preprocessing
# Кодирую оставшиеся категориальные значения. Если использовать One Hot Encoding, 
# ноутбук вылетает из-за превышения размера памяти. 

label_encoder = preprocessing.LabelEncoder()

df['status']= label_encoder.fit_transform(df['status']) 
df['propertyType']= label_encoder.fit_transform(df['propertyType']) 
df['street']= label_encoder.fit_transform(df['street'])
df['baths']= label_encoder.fit_transform(df['baths']) 
df['fireplace']= label_encoder.fit_transform(df['fireplace']) 
df['city']= label_encoder.fit_transform(df['city']) 
df['beds']= label_encoder.fit_transform(df['beds']) 
df['state']= label_encoder.fit_transform(df['state']) 
df['PrivatePool']= label_encoder.fit_transform(df['PrivatePool']) 
df['MlsId']= label_encoder.fit_transform(df['MlsId']) 
df['stories']= label_encoder.fit_transform(df['stories']) 
df['Parking']= label_encoder.fit_transform(df['Parking']) 
df['zipcode']= label_encoder.fit_transform(df['zipcode']) 
df['sqft']= label_encoder.fit_transform(df['sqft']) 
df['mls-id']= label_encoder.fit_transform(df['mls-id'])

print(df.head())


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#Выгрузим датасет для работы на локальной машине 
df.to_csv("/kaggle/working/df.csv")

# ML models

In [ ]:
X = df.drop('target',axis = 1)
y = df['target']

Разбиваю данные на test и train

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Model 1:  "Наивная" модель

В качестве нивной модели буду использовать линейную регрессию

In [ ]:

naiv_model = LinearRegression()
naiv_model.fit(X_train, y_train)
naiv_predict = naiv_model.predict(X_test)
 

print(f"Точность наивной модели по метрике MAE: {(mean_absolute_error(y_test, naiv_predict)):0.2f}")
print(f"Точность наивной модели по метрике MAPE: {(mean_absolute_percentage_error(y_test, naiv_predict)):0.2f}%")
print(f"Точность наивной модели по метрике RMSE: {(np.sqrt(mean_squared_error(y_test, naiv_predict))):0.2f}")

# # попробую логарифмировние 
naiv_model_log = LinearRegression()
naiv_model_log.fit(X_train, np.log(y_train))
naiv_predict_log = np.exp(naiv_model_log.predict(X_test))


print(f"Точность наивной модели по метрике log MAE: {(mean_absolute_error(y_test, naiv_predict_log)):0.2f}")
print(f"Точность наивной модели по метрике log MAPE: {(mean_absolute_percentage_error(y_test, naiv_predict_log)):0.2f}%")
print(f"Точность наивной модели по метрике log RMSE: {(np.sqrt(mean_squared_error(y_test, naiv_predict_log))):0.2f}")

В дальнейшем, модели с логарифмированием целевой переменной показали лучший результат, поэтому out без логарифмирования показывать не буду

# Model 2 : Random Forest

In [ ]:
rf_log = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
rf_log.fit(X_train, np.log(y_train))
predict_rf_log = np.exp(rf_log.predict(X_test))
predict_rf_x = np.exp(rf_log.predict(X_train))

print(f"The MAE train Random Forest model is : {(mean_absolute_error(y_train, predict_rf_x)):0.2f}.")
print(f"The MAE test  Random Forest model is : {(mean_absolute_error(y_test, predict_rf_log)):0.2f}.")

print(f"The MAPE train Random Forest model is : {(mean_absolute_percentage_error(y_train, predict_rf_x)):0.2f}%.")
print(f"The MAPE test  Random Forest model is : {(mean_absolute_percentage_error(y_test, predict_rf_log)):0.2f}%.")

print(f"The RMSE train Random Forest model is : {(np.sqrt(mean_squared_error(y_train, predict_rf_x))):0.2f}.")
print(f"The RMSE test Random Forest model is : {(np.sqrt(mean_squared_error(y_test, predict_rf_log))):0.2f}.")

The MAE mertic for the log Random Forest model is : 224498.49

The MAPE mertic for the log Random Forest model is : 2.20%.

The RMSE mertic for the log Random Forest model is : 1527149.66

# Model 3 : ExtraTreesRegressor

In [ ]:
etr_log = ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
etr_log.fit(X_train, np.log(y_train))
predict_etr_log = np.exp(etr_log.predict(X_test))
predict_etr_x = np.exp(etr_log.predict(X_train))

print(f"The MAE train ExtraTrees model is : {(mean_absolute_error(y_train, predict_etr_x)):0.2f}.")
print(f"The MAE test ExtraTrees model is : {(mean_absolute_error(y_test, predict_etr_log)):0.2f}.")

print(f"The MAPE train ExtraTrees model is : {(mean_absolute_percentage_error(y_train, predict_etr_x)):0.2f}%.")
print(f"The MAPE test ExtraTrees model is : {(mean_absolute_percentage_error(y_test, predict_etr_log)):0.2f}%.")

print(f"The RMSE train ExtraTrees model is : {(np.sqrt(mean_squared_error(y_train, predict_etr_x))):0.2f}.")
print(f"The RMSE test ExtraTrees model is : {(np.sqrt(mean_squared_error(y_test, predict_etr_log))):0.2f}.")

The MAE mertic for the log ExtraTrees model is : 222826.93

The MAPE mertic for the log ExtraTrees model is : 3.22%.

The RMSE mertic for the log ExtraTrees model is : 1485521.77

# Model 4 : CatBoost

In [ ]:
log_cat_boost = CatBoostRegressor(iterations = 10000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
log_cat_boost.fit(X_train, np.log(y_train),
          eval_set=(X_test, np.log(y_test)),
          verbose_eval=0,
          use_best_model=True,
          )

log_cat_boost.save_model('catboost_single_model_baseline.model')


log_predict_cat_boost = log_cat_boost.predict(X_test)
predict_cat_boost_x = np.exp(log_cat_boost.predict(X_train))

print(f"The MAE train CatBoost model: {(mean_absolute_error(y_train, predict_cat_boost_x)):0.2f}.")
print(f"The MAE test CatBoost model: {(mean_absolute_error(y_test, log_predict_cat_boost)):0.2f}")

print(f"The MAPE train CatBoost model: {(mean_absolute_percentage_error(y_train, predict_cat_boost_x)):0.2f}%.")
print(f"The MAPE test CatBoost model: {(mean_absolute_percentage_error(y_test, log_predict_cat_boost)):0.2f}%")

print(f"The RMSE train CatBoost model: {(np.sqrt(mean_squared_error(y_train, predict_cat_boost_x))):0.2f}.")
print(f"The RMSE test CatBoost model: {(np.sqrt(mean_squared_error(y_test, log_predict_cat_boost))):0.2f}")

The MAE mertic for the CatBoost model: 644162.03

The MAPE mertic for the CatBoost model: 1.00%

The RMSE mertic for the CatBoost model: 2050654.90

# Model 5: GradientBoosting

In [ ]:
gb = GradientBoostingRegressor(random_state=42)
gb.fit(X_train, np.log(y_train))
predict_gb = gb.predict(X_test)
predict_gb_x = gb.predict(X_train)

print(f"The MAE train GradientBoosting model: {(mean_absolute_error(y_train, predict_gb_x)):0.2f}.")
print(f"The MAE test GradientBoosting model: {(mean_absolute_error(y_test, predict_gb)):0.2f}")

print(f"The MAPE train GradientBoosting model: {(mean_absolute_percentage_error(y_train, predict_gb_x)):0.2f}%.")
print(f"The MAPE test GradientBoosting model: {(mean_absolute_percentage_error(y_test, predict_gb)):0.2f}%")

print(f"The RMSE train GradientBoosting model: {(np.sqrt(mean_squared_error(y_train, predict_gb_x))):0.2f}.")
print(f"The RMSE test GradientBoosting model: {(np.sqrt(mean_squared_error(y_test, predict_gb))):0.2f}")

The MAE mertic for GradientBoosting model: 644162.03

The MAPE mertic for GradientBoosting model: 1.00%

The RMSE mertic for GradientBoosting model: 2050655.03

# Model 6 : StackingRegressor

Попробую комбинацию из моделей, показавших лучшие метрики

In [ ]:
estimators = [
     ('cat_boost', CatBoostRegressor(iterations = 10000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True
                         )),
     ('gb', GradientBoostingRegressor(random_state=RANDOM_SEED))
]

sr_log = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
)

sr_log.fit(X_train, np.log(y_train))

y_pred = np.exp(sr_log.predict(X_test))

print(f"The MAE mertic for StackingRegressor model: {(mean_absolute_error(y_test, y_pred)):0.2f}")
print(f"The MAPE mertic for StackingRegressor model: {(mean_absolute_percentage_error(y_test, y_pred)):0.2f}%")
print(f"The RMSE mertic for StackingRegressor model: {(np.sqrt(mean_squared_error(y_test, y_pred))):0.2f}")

The MAE mertic for the default StackingRegressor model: 257812.28

The MAPE mertic for the default StackingRegressor model: 1.54%

The RMSE mertic for the default StackingRegressor model: 1515244.42

In [ ]:
x = PrettyTable()
x.field_names = ['metrics', "metrics_naive", "Random Forest", "ExtraTrees", 'CatBoost', 'GradientBoosting', 'Stacking' ]
x.add_row(["MAPE:", '6.74%', '2.20%', '3.22%', '1.00%', '1.00%', '1.54%'])
x.add_row(["MAE", '457497.40', '224498.49', '222826.93', '644162.03', '644162.03', '257812.28'])
x.add_row(["RMSE", '1949954.81', '1527149.66', '1485521.77', '2050654.90', '2050655.03', '1515244.42'])

In [ ]:
print(x)

# Итог

Задача: разработать сервис, который будет предсказывать стоимость домов, основываясь на истории предложений.

В ходе выполнения проекта:

Загружены данные, произведена их обработка;

Созданы новые признаки на основе имеющихся;

Построены различные модели классического ML для решения задачи регрессии;

Для каждой построенной модели получены соответствующие метрики.

Если оценивать модели по совокупности метрик, я выбираю StackingRegressor. Судя по RMSE в ней меньше расхождение между прогнозом и целевой переменной, значения MAE и RMSE ниже, чем в наивной линейной модели.
